In [ ]:
import json
import pandas as pd
from datetime import datetime, timezone
import os
import pickle

In [ ]:
start_date = datetime.fromisoformat("2018-01-01 00:00").astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
end_date = datetime.fromisoformat("2018-12-31 23:59").astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()

In [ ]:
news_path = "/Users/k2rth1k/swm/News"
ignore_files = ['.DS_Store']
news_folders = os.listdir(news_path)
article_content = []
published_time = []
for sub_folder in news_folders:
    if sub_folder in ignore_files:
        continue
    articles = os.listdir(news_path + "/" + sub_folder)
    for article in articles:
        article_path = news_path + "/" + sub_folder + "/" + article
        article_json = open(article_path)
        data = json.load(article_json)
        time_of_publish = data['published']
        time = datetime.fromisoformat(time_of_publish).astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
        published_time.append(time)
        article_content.append(data['text'].lower())

raw_data = pd.DataFrame({
    "published_time": published_time,
    'article_content': article_content
})
raw_data.drop(raw_data[(raw_data['published_time'] < start_date) | (raw_data['published_time'] > end_date)].index,
              inplace=True)

In [31]:
apple_news_data = raw_data[raw_data['article_content'].str.contains('apple|aapl')]
amazon_news_data = raw_data[raw_data['article_content'].str.contains("amazon|amzn")]
amazon_news_data = amazon_news_data[amazon_news_data['article_content'].map(lambda x: x.isascii())]
apple_news_data = apple_news_data[apple_news_data['article_content'].map(lambda x: x.isascii())]
amazon_news_data.drop_duplicates(inplace=True)
apple_news_data.drop_duplicates(inplace=True)
amazon_news_data.sort_values('published_time', ascending=True, inplace=True)
apple_news_data.sort_values('published_time', ascending=True, inplace=True)

In [32]:
apple_news_data

,published_time,article_content
16174,1.514832e+09,facebook \nciti analysts say that there is a 4...
15612,1.514855e+09,"quote: in this context avxl is a new apple, im..."
12200,1.514862e+09,2 warren buffett stocks to consider buying now...
15928,1.514862e+09,2 warren buffett stocks to consider buying now...
14085,1.514862e+09,2 warren buffett stocks to consider buying now...
...,...,...
56164,1.546321e+09,"let's talk about apple, inc. (nasdaq: aapl ). ..."
61168,1.546322e+09,big trends: gartner iaas mq is down to 6 compa...
41,1.546326e+09,- fundamental analysis - technology is apple s...
60869,1.546332e+09,"new york, oct. 23, 2018 (globe newswire) -- in..."


In [33]:
amazon_news_data

,published_time,article_content
12886,1.514876e+09,here are some things going on today in your wo...
13571,1.514876e+09,shutterstock photo\nstocks indexes opened the ...
11837,1.514876e+09,by ryan vlastelica\nto simply match the market...
13804,1.514876e+09,what happened shares of many optical networkin...
15737,1.514876e+09,by nigam arora\nthe practical way to take adva...
...,...,...
1348,1.546281e+09,the biggest benefit from living are the lesson...
2553,1.546290e+09,apple stock price\napple (nasdaq: aapl ) was v...
849,1.546306e+09,none of us has a crystal ball that shows us cl...
56164,1.546321e+09,"let's talk about apple, inc. (nasdaq: aapl ). ..."


In [34]:
with open('pkls/amazon_news_data.pkl', 'wb') as f:
    pickle.dump(amazon_news_data, f)

with open('pkls/apple_news_data.pkl', 'wb') as f:
    pickle.dump(apple_news_data, f)